In [65]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import math

In [66]:
def get_conn(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [67]:
def get_df_from_sql(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [68]:
# Usage with the actual path to the private key
SSH_required = 'Yes'
key_path = '/Users/rajatsansaniwal/Documents/tunnel-ssh .cer'
query_path = 'pickup_query.sql'
with open(query_path,'r') as file:
    query = file.read()

# Establish a connection
conn = get_conn(SSH_required, key_path)

# Retrieve data into a DataFrame
df = get_df_from_sql(SSH_required, query, key_path)

# Now you can perform further operations with the DataFrame 'df'
print(df)

Tunnel Started
Connection Made
       warehouse_id          pickuptime           awb
0                83 2024-07-06 16:49:33  GS1785205670
1                83 2024-07-06 16:49:36  GS2006391816
2                83 2024-07-06 16:49:39  GS1166207635
3                83 2024-07-06 16:49:42  GS1053787147
4                83 2024-07-06 16:49:47  GS1758550671
...             ...                 ...           ...
17426          2147 2024-07-06 12:25:24  GS1945488003
17427          2147 2024-07-06 12:25:40  GS2022778763
17428          2147 2024-07-06 12:25:42  GS1524706122
17429          2151 2024-07-06 16:46:59  GS1812158876
17430          2151 2024-07-06 16:47:04  GS1755804579

[17431 rows x 3 columns]


In [69]:
df['pickuptime'] = pd.to_datetime(df['pickuptime'])

results = []

for warehouse, group in df.groupby('warehouse_id'):
    group = group.reset_index(drop=True)

    pickup_start_time = group.loc[0, 'pickuptime']

    for i in range(1, len(group)):
        current_time = group.loc[i, 'pickuptime']
        previous_time = group.loc[i - 1, 'pickuptime']
        
        # Check the difference between current and previous pickuptime
        if (current_time - previous_time).total_seconds() > 3600:  # More than one hour
            # End the current pickup slot and start a new one
            results.append({
                'warehouse_id': warehouse,
                'pickup_start_time': pickup_start_time,
                'pickup_end_time': previous_time
            })
            pickup_start_time = current_time  # Update the start time for the new slot
    
    # Append the last slot for the warehouse
    results.append({
        'warehouse_id': warehouse,
        'pickup_start_time': pickup_start_time,
        'pickup_end_time': group.loc[len(group) - 1, 'pickuptime']
    })

# Create DataFrame from results list
results_df = pd.DataFrame(results)

# Calculate slot_duration in minutes
results_df['slot_duration'] = ((results_df['pickup_end_time'] - results_df['pickup_start_time']).dt.total_seconds() / 60).apply(math.ceil)

# Sort DataFrame by warehouse_id and pickup_start_time (if not already sorted)
results_df.sort_values(by=['warehouse_id', 'pickup_start_time'], inplace=True)

# Calculate gaps between pickup slots in minutes
results_df['gap_to_next'] = results_df.groupby('warehouse_id')['pickup_start_time'].shift(-1) - results_df['pickup_end_time']
results_df['gap_to_next'] = (results_df['gap_to_next'].dt.total_seconds() / 60)
results_df['gap_to_next'].fillna(0, inplace=True)
results_df['gap_to_next'] = (results_df['gap_to_next'].astype(int) + 1).replace(1, np.nan)

print(results_df)

     warehouse_id   pickup_start_time     pickup_end_time  slot_duration  \
0              83 2024-07-06 16:49:33 2024-07-06 16:55:17              6   
1             314 2024-07-06 01:40:19 2024-07-06 02:06:12             26   
2             314 2024-07-06 15:42:51 2024-07-06 16:06:08             24   
3             314 2024-07-06 17:37:28 2024-07-06 17:45:11              8   
4             327 2024-07-06 18:49:49 2024-07-06 18:50:15              1   
..            ...                 ...                 ...            ...   
214          2142 2024-07-06 17:53:20 2024-07-06 17:53:58              1   
215          2143 2024-07-06 18:54:56 2024-07-06 19:07:06             13   
216          2144 2024-07-06 18:54:05 2024-07-06 19:07:28             14   
217          2147 2024-07-06 12:23:09 2024-07-06 12:25:42              3   
218          2151 2024-07-06 16:46:59 2024-07-06 16:47:04              1   

     gap_to_next  
0            NaN  
1          817.0  
2           92.0  
3          

In [70]:
# Initialize an empty list to store the calculated loads
loads = []

for warehouse_id, slot in results_df.groupby('warehouse_id'):
    for index, row in slot.iterrows():
        warehouse_id = row['warehouse_id']
        slot_start_time = row['pickup_start_time']
        slot_end_time = row['pickup_end_time']

        filtered_df = df[(df['warehouse_id'] == warehouse_id) &
                         (df['pickuptime'] >= slot_start_time) &
                         (df['pickuptime'] <= slot_end_time)]

        # Count the number of awbs within the slot time range
        load = len(filtered_df)
        
        # Append the calculated load count to the list
        loads.append(load)

# Add the 'load' column to results_df
results_df['actual_load'] = loads

# Print or display the updated DataFrame with the 'load' column
print(results_df)

# Save DataFrame to Excel
excel_file = 'pickup_slots.xlsx'
results_df.to_excel(excel_file, index=False)

print(f"DataFrame saved to {excel_file}")


     warehouse_id   pickup_start_time     pickup_end_time  slot_duration  \
0              83 2024-07-06 16:49:33 2024-07-06 16:55:17              6   
1             314 2024-07-06 01:40:19 2024-07-06 02:06:12             26   
2             314 2024-07-06 15:42:51 2024-07-06 16:06:08             24   
3             314 2024-07-06 17:37:28 2024-07-06 17:45:11              8   
4             327 2024-07-06 18:49:49 2024-07-06 18:50:15              1   
..            ...                 ...                 ...            ...   
214          2142 2024-07-06 17:53:20 2024-07-06 17:53:58              1   
215          2143 2024-07-06 18:54:56 2024-07-06 19:07:06             13   
216          2144 2024-07-06 18:54:05 2024-07-06 19:07:28             14   
217          2147 2024-07-06 12:23:09 2024-07-06 12:25:42              3   
218          2151 2024-07-06 16:46:59 2024-07-06 16:47:04              1   

     gap_to_next  actual_load  
0            NaN           47  
1          817.0       

In [71]:
results = []

for warehouse, group in df.groupby('warehouse_id'):
    group = group.reset_index(drop=True)

    pickup_start_time = group.loc[0, 'pickuptime']
    awb_start_times = {group.loc[0, 'awb']: pickup_start_time}

    for i in range(1, len(group)):
        current_time = group.loc[i, 'pickuptime']
        previous_time = group.loc[i - 1, 'pickuptime']

        # Check the difference between current and previous pickuptime
        if (current_time - previous_time).total_seconds() > 3600:  # More than one hour
            # End the current pickup slot and start a new one
            results.append({
                'warehouse_id': warehouse,
                'pickup_start_time': pickup_start_time,
                'pickup_end_time': previous_time,
                'awbs': list(awb_start_times.keys())
            })
            pickup_start_time = current_time  # Update the start time for the new slot
            awb_start_times = {}

        awb_start_times[group.loc[i, 'awb']] = pickup_start_time

    # Append the last slot for the warehouse
    results.append({
        'warehouse_id': warehouse,
        'pickup_start_time': pickup_start_time,
        'pickup_end_time': group.loc[len(group) - 1, 'pickuptime'],
        'awbs': list(awb_start_times.keys())
    })

# Create DataFrame from results list
results_df = pd.DataFrame(results)

# Calculate slot_duration in minutes
results_df['slot_duration'] = ((results_df['pickup_end_time'] - results_df['pickup_start_time']).dt.total_seconds() / 60).apply(math.ceil)

# Sort DataFrame by warehouse_id and pickup_start_time (if not already sorted)
results_df.sort_values(by=['warehouse_id', 'pickup_start_time'], inplace=True)

# Calculate gaps between pickup slots in minutes
results_df['gap_to_next'] = results_df.groupby('warehouse_id')['pickup_start_time'].shift(-1) - results_df['pickup_end_time']
results_df['gap_to_next'] = (results_df['gap_to_next'].dt.total_seconds() / 60)
results_df['gap_to_next'].fillna(0, inplace=True)
results_df['gap_to_next'] = (results_df['gap_to_next'].astype(int) + 1).replace(1, np.nan)

# Explode the 'awbs' column to separate rows for each AWB
results_df_exploded = results_df.explode('awbs')

print(results_df_exploded[['warehouse_id', 'pickup_start_time', 'pickup_end_time', 'slot_duration', 'gap_to_next', 'awbs']])

     warehouse_id   pickup_start_time     pickup_end_time  slot_duration  \
0              83 2024-07-06 16:49:33 2024-07-06 16:55:17              6   
0              83 2024-07-06 16:49:33 2024-07-06 16:55:17              6   
0              83 2024-07-06 16:49:33 2024-07-06 16:55:17              6   
0              83 2024-07-06 16:49:33 2024-07-06 16:55:17              6   
0              83 2024-07-06 16:49:33 2024-07-06 16:55:17              6   
..            ...                 ...                 ...            ...   
217          2147 2024-07-06 12:23:09 2024-07-06 12:25:42              3   
217          2147 2024-07-06 12:23:09 2024-07-06 12:25:42              3   
217          2147 2024-07-06 12:23:09 2024-07-06 12:25:42              3   
218          2151 2024-07-06 16:46:59 2024-07-06 16:47:04              1   
218          2151 2024-07-06 16:46:59 2024-07-06 16:47:04              1   

     gap_to_next          awbs  
0            NaN  GS1785205670  
0            NaN  GS2

2024-07-08 18:27:18,001| ERROR   | Socket exception: Operation timed out (60)
2024-07-08 18:27:18,003| ERROR   | Socket exception: Operation timed out (60)
